### Using perplextiy Sonar model

In [65]:
import yaml
with open("config.yml", "r") as file:
    api_keys = yaml.safe_load(file)

# print(api_keys)

In [34]:
from openai import OpenAI

perplexity_api_key = api_keys['perplexity_api_key']

messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                2. Only keep the results which mentions merger or acquisitions involving F W Webb. 
                3. Remove duplicate articles based on order of appearance in search results.
                4. Summarize each event in ≤2 lines, mentioning who merged/acquired whom and when, along with any key details.
                5. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY : Short summary, Web Link to Source:
                6. Focus only on verified and relevant sources.
                7. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [35]:
client = OpenAI(api_key=perplexity_api_key, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="sonar",
    messages=messages,
)
print(response)

ChatCompletion(id='8a2c841c-5757-43e7-9213-560c50020500', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. **J.D. Johnson Co.** – Dec-2023 : F.W. Webb acquired J.D. Johnson Co., expanding its presence in Poughkeepsie, NY, with a new wholesale location offering extensive inventory and services. [5]\n2. **Rising Sun Plumbing Supply and Steinberg Plumbing Supply** – Jun-2023 : F.W. Webb acquired these two Philadelphia-based distributors, enhancing its presence in the Greater Philadelphia market. [3]\n3. **Danbury Plumbing and HVAC Supply Co.** – Mar-2023 : F.W. Webb acquired Danbury Plumbing, increasing its presence in Connecticut and expanding its services to more contractors. [1][2]\n4. **Grant Supply Co.** – Jun-2015 : F.W. Webb acquired Grant Supply, adding locations in Pleasantville and North Brunswick, NJ, to its network. [4]', role='assistant', function_call=None, tool_calls=None), delta={'role': 'assistant', 'content': ''})], 

In [38]:
# chat completion without streaming
response_pro = client.chat.completions.create(
    model="sonar-pro",
    messages=messages,
)
print(response_pro)

ChatCompletion(id='fea776d2-e3e7-4629-a68a-060c0a36dfb9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Danbury Plumbing and HVAC Supply Co. - Mar-2024: F.W. Webb acquired this Western Connecticut wholesale distributor, expanding its presence in the state. https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/\n\n2. J.D. Johnson Co. - Dec-2023: F.W. Webb acquired J.D. Johnson Co. in Poughkeepsie, NY, opening a new wholesale location and expanding its presence in New York state. https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.\n\n3. Rising Sun Plumbing Supply and Steinberg Plumbing Supply - Jun-2023: F.W. Webb acquired these two Pennsylvania-based plumbing, heating, and industrial supply distributors, establishing its third and fourth wholesale locations in the state. https://www.mdm.com/news/o

### Output from Sonar Search and the citations

In [39]:
response_dict = response.to_dict()
print(response_dict['choices'][0]['message']['content'])

1. **J.D. Johnson Co.** – Dec-2023 : F.W. Webb acquired J.D. Johnson Co., expanding its presence in Poughkeepsie, NY, with a new wholesale location offering extensive inventory and services. [5]
2. **Rising Sun Plumbing Supply and Steinberg Plumbing Supply** – Jun-2023 : F.W. Webb acquired these two Philadelphia-based distributors, enhancing its presence in the Greater Philadelphia market. [3]
3. **Danbury Plumbing and HVAC Supply Co.** – Mar-2023 : F.W. Webb acquired Danbury Plumbing, increasing its presence in Connecticut and expanding its services to more contractors. [1][2]
4. **Grant Supply Co.** – Jun-2015 : F.W. Webb acquired Grant Supply, adding locations in Pleasantville and North Brunswick, NJ, to its network. [4]


In [40]:
for i,x in enumerate(response_dict['citations']):
    print(f"{i+1}: {x}")
# print(response_dict['citations'])

1: https://hvacinsider.com/f-w-webb-company-acquires-danbury-plumbing-and-hvac-supply-co/
2: https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/
3: https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/
4: https://www.supplyht.com/articles/98609-fw-webb-acquires-grant-supply
5: https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.


### Output from Sonar Pro 

In [41]:
response_pro_dict = response_pro.to_dict()
print(response_pro_dict['choices'][0]['message']['content'])

1. Danbury Plumbing and HVAC Supply Co. - Mar-2024: F.W. Webb acquired this Western Connecticut wholesale distributor, expanding its presence in the state. https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/

2. J.D. Johnson Co. - Dec-2023: F.W. Webb acquired J.D. Johnson Co. in Poughkeepsie, NY, opening a new wholesale location and expanding its presence in New York state. https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.

3. Rising Sun Plumbing Supply and Steinberg Plumbing Supply - Jun-2023: F.W. Webb acquired these two Pennsylvania-based plumbing, heating, and industrial supply distributors, establishing its third and fourth wholesale locations in the state. https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/

4. Grant Supply Co. - Jun-2015: F.W. Webb acquired Grant Supply Co., add

In [42]:
for i,x in enumerate(response_pro_dict['citations']):
    print(f"{i+1}: {x}")
# print(response_dict['citations'])

1: https://hvacinsider.com/f-w-webb-company-acquires-danbury-plumbing-and-hvac-supply-co/
2: https://edisciplinas.usp.br/pluginfile.php/5547326/mod_resource/content/1/Willmer,%20Stone%20%20Johsnton_Environmental-physiology-of-animals.pdf
3: https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/
4: https://saude.ufpr.br/medintegrada/wp-content/uploads/sites/40/2021/01/jama-the-rational-clinical-examination.pdf
5: https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/
6: https://www.micromeritics.com/downloads/eBook/Analytical-Methods-in-Fine-Particle-Technology.pdf
7: https://www.supplyht.com/articles/98609-fw-webb-acquires-grant-supply
8: https://cetesb.sp.gov.br/aguasinteriores/wp-content/uploads/sites/33/2017/04/Handbook-on-the-Toxicology-of-Metals.pdf
9: https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J

### With Gemini

In [47]:
import google.generativeai as genai
gemini_api_key = api_keys['gemini_api_key']
genai.configure(api_key=gemini_api_key)

In [59]:
model_15flash = genai.GenerativeModel(
    "models/gemini-1.5-flash", system_instruction=messages[0]['content'], tools='google_search_retrieval'
)
response_15flash = model_15flash.generate_content(messages[1]['content'])
print(response_15flash.text)

1. F.W. Webb Company acquires J.D. Johnson Co. – Dec-2023 : F.W. Webb acquired J.D. Johnson Co., opening a new wholesale location in Poughkeepsie, NY.  This acquisition expands F.W. Webb's presence in the Northeast.


2. F.W. Webb acquires Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023 : F.W. Webb acquired two Pennsylvania-based plumbing distributors, Rising Sun and Steinberg, expanding its market reach in the Philadelphia area.  Both companies will operate under the F.W. Webb name.


3. F.W. Webb acquires Danbury Plumbing and HVAC Supply Co. – Mar-2021 : F.W. Webb acquired Danbury Plumbing and HVAC Supply Co. in Western Connecticut. The acquired location became an F.W. Webb location, and Creative Bath opened as a Frank Webb Home retail showroom.





In [61]:
model_15pro = genai.GenerativeModel(
    "models/gemini-1.5-pro", system_instruction=messages[0]['content'], tools='google_search_retrieval'
)
response_15pro = model_15pro.generate_content(messages[1]['content'])
print(response_15pro.text)

1. J.D. Johnson Co. – Dec-2023: F.W. Webb acquired J.D. Johnson Co. and opened a new wholesale location in Poughkeepsie, NY.
2. Rising Sun Plumbing Supply & Steinberg Plumbing Supply – Jun-2023: F.W. Webb acquired these two Pennsylvania-based plumbing distributors.
3. Danbury Plumbing and HVAC Supply Co. – Mar-2021: F.W. Webb acquired this Western Connecticut wholesale distributor.





In [68]:
# !pip install -U -q google-genai

In [79]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [80]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023: F.W. Webb acquired J.D. Johnson Co., expanding its wholesale presence in Poughkeepsie, New York. Web Link to Source: [https://supplyht.com/articles/f-w-webb-acquires-j-d-johnson-co/](https://supplyht.com/articles/f-w-webb-acquires-j-d-johnson-co/)
2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023: F.W. Webb acquired both Rising Sun Plumbing Supply and Steinberg Plumbing Supply, expanding its reach in the Philadelphia metro area. Web Link to Source: [https://www.inddist.com/mergers-acquisitions/news/21252288/fw-webb-acquires-pennsylvania-plumbing-suppliers](https://www.inddist.com/mergers-acquisitions/news/21252288/fw-webb-acquires-pennsylvania-plumbing-suppliers)
3.  VJ Stanley and Stritt & Priebe – Jan-2022: F.W. Webb acquired both VJ Stanley and Stritt & Priebe, further expanding its operations. Web Link to Source: [https://tracxn.com/d/acquisitions/f-w-webb__1000000000000000000000000000000000000000000000000000000000000000](https

In [ ]:
# Few Links doesnt seems appropriate and working. Restructuring the prompt.

In [97]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Prioritize results where more details about the event are known like the nature of acquisition, date, benefits etc.
                4. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                5. Only keep the results which mentions merger or acquisitions involving F W Webb. 
                6. Remove duplicate articles based on order of appearance in search results.
                7. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Web address to this item from search results
                8. Focus only on verified and relevant sources.
                9. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [98]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [99]:
print(response_20flash.text)

1.  J.D. Johnson Co. – DEC-2023
    F.W. Webb acquired J.D. Johnson Co., a business that has served the community for more than 150 years. This acquisition led to the opening of a new F.W. Webb wholesale location in Poughkeepsie, New York.
    https://www.supplyht.com/articles/103017-fw-webb-acquires-jd-johnson-co

2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – JUN-2023
    F.W. Webb acquired Rising Sun Plumbing Supply in Philadelphia and Steinberg Plumbing Supply in Elkins Park, both of which have been serving the Philadelphia metro area for decades. These acquisitions expanded F.W. Webb's operations in Pennsylvania.
    https://www.inddist.com/mergers-acquisitions/news/22528288/fw-webb-acquires-pennsylvania-plumbing-suppliers

3.  VJ Stanley and Stritt & Priebe – JAN-2022
    F.W. Webb acquired VJ Stanley and Stritt & Priebe. The specific details of these acquisitions were not disclosed.
    https://tracxn.com/d/acquisitions/f-w-webb

4.  Danbury Plumbing & Supply – M

### Another way of prompting

In [100]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Web address to this item from search results
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [101]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [102]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
    F.W. Webb acquired J.D. Johnson Co., a Poughkeepsie, NY based company that has served the community for over 150 years. This acquisition allowed F.W. Webb to open a new wholesale location in Poughkeepsie.
    https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsCk5X9BC2xL4ACUV8NdYflovw5jbUr8AkDaNfsDcXksqROAipsFyX8MHz6aU_FGVXiaa2_Xspd6whVHU70shGm046ngmIYYKhHXr1iYKgb-T8URgJaNFtHUN3a0BEvSgzEvQ8aHoH3-Sy-9VRjxYvKzHUF32EC-Q-CezRSjtGCDAGB

2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023
    F.W. Webb acquired two Pennsylvania-based plumbing distributors, Rising Sun Plumbing Supply and Steinberg Plumbing Supply, expanding its presence in the Greater Philadelphia area. Both companies will operate under the F.W. Webb umbrella.
    https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsACqdedgbU_GTv222IBodYBjWADVvI0vro3-oVTvYLcvPpUC1ymSCnlbverJcaf-Sav3If1nr4nqacariUn6g6pPU4Z2lthzMka_JGtrNNPbTudzeEmIv2st9Zi

In [103]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Public Web address to this item
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [104]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [105]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
    F.W. Webb acquired J.D. Johnson Co., a plumbing, heating, and industrial supply distributor with a history of over 150 years. This acquisition allowed F.W. Webb to expand its presence in Poughkeepsie, NY.
    [https://supplyht.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu_aU1xi5yG3VjNNYf1LH49ncggPqJdPIDUAX5sBzPqVQmYhBhAYlt-YpryQqE2AfODB-1yUyOIVebD7mnRpeJ2Bie10MbZjVVxYAHRAfLNHL0WFGvvCzPxJJcPeCmI274L51Q6_v1YP2G3ItWDsfzuX06iKQwYlQPLjqgTuBhDovM=)
2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023
    F.W. Webb acquired two Pennsylvania-based plumbing distributors, Rising Sun Plumbing Supply and Steinberg Plumbing Supply, expanding its reach in the Greater Philadelphia area. Both companies now operate under the F.W. Webb umbrella.
    [https://mdm.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvWB2A7Y2J5gAqr-EoynWD7VOi5j2bkjJSMOt7mw1hJ3J1frq3-31QRAke3fKPGECr0yZukMJNCKgG4z3Ocrupd

In [106]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Link to this item from the original source
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [107]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [108]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
F.W. Webb acquired J.D. Johnson Co., a Poughkeepsie, NY based HVAC supplier, expanding their presence in the New York market. The acquisition was completed in early December 2023.
[https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv_T6sv0VbNSD8NfoaUVk-uGgAeqFzfK9aNxicOPD1CCUI0Bw8dtIQtbYHqL-1AuBDaTvHnpwEx4wB6TXiA9uR3GxgR2x8D7mx_tjX4S2R39OALQXczrLJpRd4e9Q7Gcch0Kgy-PpvLzqrtHAAAlZjFHTPO6UYPixvtcHwoLTy0YZcU](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv_T6sv0VbNSD8NfoaUVk-uGgAeqFzfK9aNxicOPD1CCUI0Bw8dtIQtbYHqL-1AuBDaTvHnpwEx4wB6TXiA9uR3GxgR2x8D7mx_tjX4S2R39OALQXczrLJpRd4e9Q7Gcch0Kgy-PpvLzqrtHAAAlZjFHTPO6UYPixvtcHwoLTy0YZcU)

2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023
F.W. Webb acquired two Pennsylvania-based plumbing distributors, Rising Sun Plumbing Supply and Steinberg Plumbing Supply, in June 2023. These acquisitions expanded F.W. Webb's reach in the Philadelphia area.
[https://vertex

In [112]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                Scan the first 50 search results.
                1. Keep only publicly accessible sources. Avoid paid/proprietary databases where full access is restricted.
                2. Retain results where F W Webb is mentioned and the source has a published date, ensuring data is from the last 10 years.
                3. Keep only results explicitly mentioning mergers or acquisitions involving F W Webb.
                4. Remove duplicate articles based on order of appearance in search results. Proceed with analysis only on the filtered list.
                5. Exclude any result where the full, original web link is not available.
                6. Create a numbered list (max 5 items), sorted by date (latest to oldest), using this format:
                    Company Acquired/Merged – MMM-YYYY
                    A summary in 2 lines or fewer, mentioning who merged/acquired whom and when, along with any key details.
                    Provide the full, original web link to the article or webpage. Do not use shortened, tracking, or search result links.
                7. Focus only on verified and relevant sources.
                8. If no eligible results are found, return only "No" with no additional text.

                '''
        ),
    },
]


In [113]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [114]:
print(response_20flash.text)

1.  J.D. Johnson Co. – DEC-2023
    F.W. Webb acquired J.D. Johnson Co., a business serving the Poughkeepsie, NY community for over 150 years, to expand its presence in the area. The acquisition was completed on December 4, 2023, and the location now operates as an F.W. Webb wholesale location.
    [https://www.supplyht.com/articles/103024-fw-webb-acquires-jd-johnson-co](https://www.supplyht.com/articles/103024-fw-webb-acquires-jd-johnson-co)

2.  Rising Sun Plumbing Supply – JUN-2023
    F.W. Webb acquired Rising Sun Plumbing Supply, a Philadelphia-based distributor, to expand its reach in the greater Philadelphia market. The acquisition was announced on June 5, 2023, and the location now operates under the F.W. Webb umbrella.
    [https://www.mdm.com/news/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/](https://www.mdm.com/news/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/)

3.  Steinberg Plumbing Supply – JUN-2023
   

Search Result Snippets: LLMs often rely on the snippets provided by the search engine, which are short summaries of the web page content.  The actual URL might be associated with the entire page, but the LLM might only have access to the snippet and its associated (sometimes truncated or redirected) URL.

Dynamic Content: Web pages are dynamic.  The content that the search engine indexed might be different from what's on the page when the LLM tries to access it.  This can lead to broken links or content mismatches.

Paywalls and Restrictions: Even if the initial search result indicated a publicly accessible page, the website might have implemented changes (paywalls, registration requirements) that restrict access.  The LLM doesn't have a way to "log in" or bypass these restrictions.

URL Shortening/Tracking: Some websites use URL shorteners or tracking links, which can make it difficult to determine the actual destination URL.  The LLM might present the shortened/tracked link, which could be less reliable.

LLM Interpretation: Even with the correct URL, the LLM might not always present it correctly in the response.  Sometimes, it can get confused with formatting or other elements in the search result.

Strategies to Improve Link Reliability:

Explicitly Request Links: In your prompt, be very clear that you need the original, fully functional web links.  For example:  "Provide the full, original web link to each source.  Do not use shortened links or links to search result pages.  I need the direct link to the article or web page itself."  The more explicit you are, the better the chances.

Limit Snippet Dependence (If Possible):  If the API allows it, explore options to make the LLM rely less on snippets and more on the actual page content (though this might be more computationally expensive).  This is not always an available option but is worth investigating in the API documentation.

Post-Processing:  The most reliable approach is often to post-process the LLM's output.  This means taking the links provided by the LLM and then programmatically checking them for validity.  You can use a library like requests in Python to make HTTP requests to the URLs and check the status codes (200 OK, 404 Not Found, etc.).  This allows you to filter out broken links or redirect to paywalls.

Specify Acceptable Domains:  If you're looking for information from specific, reputable sources, you can tell the LLM to prioritize those domains.  This can increase the likelihood of getting reliable links.  For example, "Focus on sources from reputable news organizations like the New York Times, Reuters, or the Wall Street Journal."

Iterative Refinement: If you get a broken link, try rephrasing your query or requesting the information again.  Sometimes, the LLM might provide a different link on a subsequent attempt.

Human Verification: For critical information, always verify the links and the content yourself.  LLMs are powerful tools, but they are not perfect.  Human oversight is still essential.

Example Post-Processing (Python):